In [ ]:
#  jupyter lab --allow-root --ip=0.0.0.0 --no-browser

In [111]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import datetime
import calendar
import xmltodict 


In [112]:

class Lesson:
  def __init__(self, name, start, end, instructor, activity):
    self.name = name
    self.start = start
    self.end = end
    self.instructor = instructor
    self.activity = activity

  def print(self):
    result = self.activity + ' ' + self.name + ' start: ' + self.start + ', end: ' + self.end + ' (' + self.instructor + ')'
    print(result)



In [113]:
class Studio:
  def __init__(self, name, login):
    self.name = name
    self.login = login

  def print():
      print('\n***** ' + self.name + ' *****')

  def login():
      pass

  def todays_lessons(self, driver):
      pass

  def weeks_lessons(self, driver):
      pass
    
      

In [114]:
class DaysLessons:
  def __init__(self, date, lessons):
    self.lessons = lessons
    self.date = date
    pass


  def print(self):
    print(calendar.day_name[self.date.weekday()] + ' ' +  self.date.strftime('%d.%m.%Y'))
    
    if not self.lessons:
        print('No lessons for this date')
        return
        
    for lesson in self.lessons:
        lesson.print()

        
    

In [154]:
class WeeksLessons:
  def __init__(self, studio, lessons):
    self.weeks_lessons = lessons
    self.studio = studio
    pass


  def print(self):
    print('\n***** ' + self.studio + ' *****')
    for day in self.weeks_lessons:
        day.print()        
        
    

In [155]:
class Studio_inrs(Studio):
  def __init__(self, name, login, todays_lesson):
    self.name = name
    self.login = login
    self.todays_lesson = todays_lesson


  def inrs_login(self, driver, login_data):
    url = login_data['url']
        
    driver.get(url)
    prihlasit_elem = driver.find_element(By.XPATH, "//a[contains(text(), 'Přihlásit')]")
    prihlasit_elem.click()
    
    time.sleep(1) 
    
    username_elem = driver.find_element(By.NAME, "username") 
    username_elem.send_keys(login_data['username'])
    
    psswd_elem = driver.find_element(By.NAME, "password")
    psswd_elem.send_keys(login_data['password'])
    
    prihlasit_accept_elem = driver.find_element(By.XPATH, "//button[contains(text(), 'Přihlásit')]")
    prihlasit_accept_elem.click()

    if self.name == 'Decadance':
        time.sleep(5)
        prihlasit_dokoncit_elem = driver.find_element(By.XPATH, "//a[contains(text(), 'Dokončit přihlášení')]")
        prihlasit_dokoncit_elem.click()


  def inrs_get_todays_lessons(self, driver):
    today = datetime.today().strftime('%Y-%m-%d')

    todays_lessons = []
    rooms = driver.find_elements(By.CLASS_NAME, "btn-mistnost-vyber")
    for room in rooms:
        room.click()
        time.sleep(2)
        todays_lessons += self.inrs_get_room_lessons(date)

    return todays_lessons

  def inrs_get_days_lessons(self, driver, date):
    todays_lessons = []
    rooms = driver.find_elements(By.CLASS_NAME, "btn-mistnost-vyber")
    for room in rooms:
        room.click()
        time.sleep(2)
        todays_lessons += self.inrs_get_room_lessons(date)

    return todays_lessons



  def insr_get_weeks_lessons(self, driver):
      lessons = []
      for i in range(7):
        lessons.append([])

      rooms = driver.find_elements(By.CLASS_NAME, "btn-mistnost-vyber")
      for room in rooms:
        room.click()
        time.sleep(2)
        for i in range(7):
          date = datetime.datetime.now() + datetime.timedelta(days=i)
          date_str = date.strftime('%Y-%m-%d')
          lessons[i] += self.inrs_get_room_lessons(date_str)

      todays_lessons = []
      for i in range(7):
        date = datetime.datetime.now() + datetime.timedelta(days=i)
        todays_lessons.append(DaysLessons(date, lessons[i]))

      return WeeksLessons(self.name, todays_lessons)
          

    

  def inrs_get_room_lessons(self, today, available_only=False):
    today_div = driver.find_element(By.CLASS_NAME, "lekce-wrapper-" + today)
    today_divs = today_div.find_elements(By.CLASS_NAME, "lekce-wrapper-vypis")
    
    todays_lessons = []
    
    for lesson in today_divs:
        try:
            if not available_only:
              # this should throw exception if doesn't exist / it should be skipped 
              reservation = lesson.find_element(By.CLASS_NAME, "rezervace_info_do_kalendare")
    
            time_slot = lesson.find_element(By.CLASS_NAME, "lekce-telo-cas")
    
            start = lesson.find_element(By.CLASS_NAME, "cas-od").text
            end = time_slot.find_element(By.CLASS_NAME, "cas-do").text
    
            activity =  lesson.find_element(By.CLASS_NAME, "lekce-telo-aktivita").text
            name = lesson.find_element(By.CLASS_NAME, "lekce-telo-nazev").text
            instructor = lesson.find_element(By.CLASS_NAME, "lekve-telo-instruktor").text

            todays_lessons.append(Lesson(name, start, end, instructor, activity))

        except Exception:
            continue

    return todays_lessons

In [156]:
class Decadance(Studio_inrs):
  def __init__(self):
    self.name = 'Decadance'

  def login(self, driver, login_data):
    self.inrs_login(driver, login_data)

  def todays_lessons(self, driver):
    return self.inrs_get_todays_lessons(driver)



class Citadela(Studio_inrs):
  def __init__(self):
    self.name = 'Citadela'
      
  def login(self, driver, login_data):
    self.inrs_login(driver, login_data)

  def todays_lessons(self, driver):
      return self.inrs_get_todays_lessons(driver)



In [157]:
def load_login_data():
  with open('./studios_data.xml', 'r', encoding='utf-8') as file: 
    xml_data = file.read() 
 
    return xmltodict.parse(xml_data) 
  

In [158]:

def todays_lessons(studios):
  # todays lessons
  login_data = load_login_data()

  lessons = []
  for studio in studios:
    studio.login(driver, login_data['studios'][studio.name])
    time.sleep(2)
    lessons += studio.todays_lessons(driver)

  print('Printing lessons:')
  for lesson in lessons:
    lesson.print()
    


In [159]:
def weeks_lessons(studios):

  login_data = load_login_data()

  weeks_lessons = []
  # weeeks lessons
  for studio in studios:
    studio.login(driver, login_data['studios'][studio.name])
    time.sleep(2)
    weeks_lessons.append(studio.insr_get_weeks_lessons(driver))
    

  print('Printing lessons:')
  for lesson in weeks_lessons:
    lesson.print()
    



In [160]:
driver = webdriver.Chrome()

decadance = Decadance()
citadela = Citadela()


studios = [decadance, citadela]

weeks_lessons(studios)


driver.close()


Printing lessons:

***** Decadance *****
Monday 26.08.2024
No lessons for this date
Tuesday 27.08.2024
No lessons for this date
Wednesday 28.08.2024
OPEN CLASS AERIAL HOOP BEGINNERS start: 19:05, end: - 20:05 (Eva Sch.)
Thursday 29.08.2024
OPEN CLASS POLE DANCE INTERMEDIATE start: 16:30, end: - 17:30 (Eva Sch.)
Friday 30.08.2024
No lessons for this date
Saturday 31.08.2024
No lessons for this date
Sunday 01.09.2024
No lessons for this date

***** Citadela *****
Monday 26.08.2024
No lessons for this date
Tuesday 27.08.2024
No lessons for this date
Wednesday 28.08.2024
No lessons for this date
Thursday 29.08.2024
No lessons for this date
Friday 30.08.2024
No lessons for this date
Saturday 31.08.2024
No lessons for this date
Sunday 01.09.2024
No lessons for this date
